Русский текст: Вишневский стал довольным. Прекрасное далёко будь широким \
Мороженое мясо разморозили. Мороженное в ГУМе вкусное. НКРЯ — хороший корпус. Пила \
выглядела острой, Лена пила компот из сухофруктов. Ванная — помещение в квартире. \
Ванная комната. Глокой куздры развихры кудрячились по белесе. Спасло чудо техники. \
Спасли чудом техники. Ток не бей по лицу, пожалуйста. Ток ударил электрика. Бывший КГБшник, \
нынешний президент. Здравствуйте, дорогой Мартин Алексеич! Дорогая, пойдем уже отсюда? \
Спел песню. Посмотри, как спел этот фрукт. Смешное животное смешно смеялось. \
Мне кажется, уже достаточно много трудных для POS теггинга случаев и \
я не знаю, что дальше писать и откуда брать текст, но до ста слов дойти хочется.'

Неоднозначные моменты для тэггинга: Вишневский -- ADJ или NOUN? Далёко -- как существительное и наречие; Мороженое мясо и мороженое -- конверсивы; Острая пила и Лена пила сок -- омонимы; Ванная как прилигательное и существительное; Ток -- как частица только и ток как электрический ток; Дорогой Мартин Алексеич и Дорогая -- конверсивы; Спел фрукт и спел песню -- омонимы; POS -- английская вставка. Также есть аббревиатуры (ГУМ, НКРЯ) и выдуманные слова, имеющие морфологию частей речи (например глокая -- прил,куздра сущ и т.д.)

Английский текст: Dr. Young healed young child. Lying on the bed is good after a hard day. \
He is lying. White people and black people. This sentence is put here for more words \
and diversity. Mr. White works at the school. He walked cat for a catwalk. He lights \
the lanterns. Lights were bright in the dark. POS tagging is quite interesting thing. \
Walking in the rain is my hobby. He was walking in the rain. I play my music. \
Russian indie bands are super. It is raining. Streets are full of different people. \
NRA has their people in the senate of the US.

Неоднозначные моменты: конверсивы в собственных именах и прилагательных (Dr. Young и young, Mr. White); омонимы (lying как глагол и существительное; lights как форма глагола и существительное; walking как форма глагола и существительное); Аббревиуатуры и имена собственные (NRA, US).

Разметку текстов можно найти в функциях проверки accuracy ниже. Сразу отмечу, что к такому стандарту я и привожу output'ы разных тэггеров.

Самым эффективным русским тэггором оказался mystem, а английским -- nltk. К слову, flair слова с дефисом за два слова считает, поэтому в английском варианте текста "POS-tagging" пришлось превратить в "POS tagging".

In [112]:
import navec
import string
import re
import spacy
import nltk
from flair.data import Sentence
from flair.models import SequenceTagger
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
from pymorphy2 import MorphAnalyzer
from navec import Navec
from slovnet import NER
from sklearn.metrics import accuracy_score
from ipymarkup import show_span_ascii_markup as show_markup
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger, 
    PER,
    NamesExtractor,
    Doc
)
nltk.download('averaged_perceptron_tagger')
segmenter = Segmenter()
morph = MorphAnalyzer()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
m = Mystem()
tagger = SequenceTagger.load('pos-multi')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kirillkonca1/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


2020-10-15 01:00:05,444 loading file /Users/kirillkonca1/.flair/models/pos-multi-v0.1.pt


In [113]:
text = 'Вишневский стал довольным. Прекрасное далёко будь широким \
Мороженое мясо разморозили. Мороженное в ГУМе вкусное. НКРЯ — хороший корпус. Пила \
выглядела острой, Лена пила компот из сухофруктов. Ванная — помещение в квартире. \
Ванная комната. Глокой куздры развихры кудрячились по белесе. Спасло чудо техники. \
Спасли чудом техники. Ток не бей по лицу, пожалуйста. Ток ударил электрика. Бывший КГБшник, \
нынешний президент. Здравствуйте, дорогой Мартин Алексеич! Дорогая, пойдем уже отсюда? \
Спел песню. Посмотри, как спел этот фрукт. Смешное животное смешно смеялось. \
Мне кажется, уже достаточно много трудных для POS теггинга случаев и \
я не знаю, что дальше писать и откуда брать текст, но до ста слов дойти хочется.'

text_eng = 'Dr. Young healed young child. Lying on the bed is good after a hard day. \
He is lying. White people and black people. This sentence is put here for more words \
and diversity. Mr. White works at the school. He walked cat for a catwalk. He lights \
the lanterns. Lights were bright in the dark. POS tagging is quite interesting thing. \
Walking in the rain is my hobby. He was walking in the rain. I play my music. \
Russian indie bands are super. It is raining. Streets are full of different people. \
NRA has their people in the senate of the US.'


def pymorph(text):
    tags = []
    for i in text.split():
        if i != '—':
            p = morph.parse(i.translate(str.maketrans('', '', string.punctuation)))[0]
            if p.tag.POS == 'ADJF':
                tags.append('ADJ')
            elif p.tag.POS == 'ADJS':
                tags.append('ADJ')
            elif p.tag.POS == 'COMP':
                tags.append('ADV')
            elif p.tag.POS == 'INFN':
                tags.append('VERB')
            elif p.tag.POS == 'PRTF':
                tags.append('VERB')
            elif p.tag.POS == 'GRND':
                tags.append('VERB')
            elif p.tag.POS == 'PRTS':
                tags.append('VERB')
            elif p.tag.POS == 'PRCL':
                tags.append('PART')
            elif p.tag.POS == 'PRED':
                tags.append('ADV')
            elif p.tag.POS == 'ADVB':
                tags.append('ADV')
            elif p.tag.POS == 'NPRO':
                tags.append('PRO')
            elif p.tag.POS == None:
                tags.append('None')
            else:
                tags.append(p.tag.POS)
    return tags

def mystem(text):
    ana = m.analyze(text)
    tags = []
    for word in ana:
        try:
            if 'analysis' in word:
                gr = word['analysis'][0]['gr']
                pos = gr.split('=')[0].split(',')[0]
                if pos == 'S':
                    tags.append('NOUN')
                elif pos == 'V':
                    tags.append('VERB')
                elif pos == 'A':
                    tags.append('ADJ')
                elif pos == 'ADVPRO':
                    tags.append('ADV')
                elif pos == 'NUM':
                    tags.append('NUMR')
                elif pos == 'SPRO':
                    tags.append('PRO') 
                elif pos == 'PR':
                    tags.append('PREP') 
                elif pos == 'APRO':
                    tags.append('ADJ')
                elif pos == 'ANUM':
                    tags.append('ADJ')
                else:
                    tags.append(pos)
        except IndexError: #except потому что при парсинге английского POS крашится, так как там analysis пустой
            tags.append('None')
    return tags

        
def nat(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    tags = []
    for i in doc.tokens:
        if i.pos != 'PUNCT':
            if i.pos == 'PROPN':
                tags.append('NOUN')
            elif i.pos == 'AUX':
                tags.append('VERB')
            elif i.pos == 'SCONJ':
                tags.append('CONJ')
            elif i.pos == 'CCONJ':
                tags.append('CONJ')
            elif i.pos == 'DET':
                tags.append('PRO')
            elif i.pos == 'ADP':
                tags.append('PREP')
            else:
                tags.append(i.pos)
    return tags

def spacy_tag(text):
    tags = []
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    for token in doc:
        if token.pos_ != 'PUNCT':
            if token.pos_ == 'PROPN':
                tags.append('NOUN')
            elif token.pos_ == 'ADP':
                tags.append('PREP')
            elif token.pos_ == 'CCONJ':
                tags.append('CONJ')
            elif token.pos_ == 'AUX':
                tags.append('VERB')
            else:
                tags.append(token.pos_)           
    return tags

def flair_tag(text):
    tags = []
    sentence = Sentence(text)
    tagger.predict(sentence)
    tags_string = sentence.to_tagged_string()
    search = re.findall('\<\w+\>', tags_string)
    for tag in search:
        tag_n = re.sub('[^A-Za-z]+', '', tag)
        if tag_n != 'PUNCT':
            if tag_n == 'PROPN':
                tags.append('NOUN')
            elif tag_n == 'AUX':
                tags.append('VERB')
            elif tag_n == 'ADP':
                tags.append('PREP')
            elif tag_n == 'CCONJ':
                tags.append('CONJ')
            elif tag_n == 'PRO':
                tags.append('PRON')
            else:
                tags.append(tag_n)
    return tags

def nltk_tags(text):
    tags = []
    tokenize = word_tokenize(text)
    nltk.pos_tag(tokenize)
    for i in nltk.pos_tag(tokenize):
        if i[1] != '.':
            if i[1] == 'CC':
                tags.append('CONJ')
            elif i[1] == 'DT':
                tags.append('DET')
            elif i[1] == 'NN' or i[1] == 'NNS' or i[1] == 'NNPS' or i[1] == 'NNP':
                tags.append('NOUN')
            elif i[1] == 'JJ' or i[1] == 'JJR' or i[1] == 'JJS':
                tags.append('ADJ')
            elif i[1] == 'IN':
                tags.append('PREP')
            elif i[1] == 'PRP' or i[1] == 'PRP$':
                tags.append('PRO')
            elif i[1] == 'RB':
                tags.append('ADV')
            elif i[1] == 'VB' or i[1] == 'VBD' or i[1] == 'VBG' \
            or i[1] == 'VBN' or i[1] == 'VBP' or i[1] == 'VBZ':
                tags.append('VERB')
    return tags

def accuracy_check_rus(tags):
    gold = ['NOUN', 'VERB', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'ADJ',
            'ADJ', 'NOUN', 'VERB', 'NOUN', 'PREP', 'NOUN', 'ADJ', 'NOUN', 
            'ADJ', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'VERB', 'NOUN',
            'PREP', 'NOUN', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'ADJ', 'NOUN',
            'ADJ', 'NOUN', 'NOUN', 'VERB', 'PREP', 'NOUN', 'VERB', 'NOUN',
            'NOUN', 'VERB', 'NOUN', 'NOUN', 'PART', 'PART', 'VERB', 'PREP',
            'NOUN', 'PART', 'NOUN', 'VERB', 'NOUN', 'ADJ', 'NOUN', 'ADJ',
            'NOUN', 'VERB', 'ADJ', 'NOUN', 'NOUN', 'ADJ', 'VERB', 'PART',
            'ADV', 'VERB', 'NOUN', 'VERB', 'PART', 'ADJ', 'PRO', 'NOUN',
            'ADJ', 'NOUN', 'ADV', 'VERB', 'PRO', 'VERB', 'PART', 'ADJ',
            'NUM', 'ADJ', 'PREP', 'NOUN', 'NOUN', 'NOUN', 'CONJ', 'PRO',
            'PRCL', 'VERB', 'CONJ', 'ADV', 'VERB', 'CONJ', 'ADV', 'VERB',
            'NOUN', 'CONJ', 'PREP', 'NUMR', 'NOUN', 'VERB', 'VERB']
    print("Accuracy: %.4f" % accuracy_score(tags, gold))

def accuracy_check_eng(tags):
    gold = ['NOUN', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'NOUN', 'PREP', 'DET', 'NOUN', 'VERB', 'ADJ', 'PREP', 
            'DET', 'ADJ', 'NOUN', 'PRO', 'VERB', 'VERB', 'ADJ', 'NOUN', 'CONJ', 'ADJ', 'NOUN',
            'PRO', 'NOUN', 'VERB', 'VERB', 'ADV', 'PREP', 'ADJ', 'NOUN', 'CONJ', 'NOUN', 'NOUN',
            'NOUN', 'VERB', 'PREP', 'DET', 'NOUN', 'PRO', 'VERB', 'NOUN', 'PREP', 'DET', 'NOUN',
            'PRO', 'VERB', 'DET', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'PREP', 'DET', 'NOUN', 'NOUN',
            'NOUN', 'VERB', 'ADV', 'ADJ', 'NOUN', 'NOUN', 'PREP', 'DET', 'NOUN', 'VERB', 'PRO', 'NOUN',
            'PRO', 'VERB', 'VERB', 'PREP', 'DET', 'NOUN', 'PRO', 'VERB', 'PRO', 'NOUN', 'ADJ',
            'ADJ', 'NOUN', 'VERB', 'ADJ', 'PRO', 'VERB', 'VERB', 'NOUN', 'VERB', 'ADJ', 'PREP', 'ADJ',
            'NOUN', 'NOUN', 'VERB', 'PRO', 'NOUN', 'PREP', 'DET', 'NOUN', 'PREP', 'DET', 'NOUN']
    print("Accuracy: %.4f" % accuracy_score(tags, gold))
    
def adj_noun(text):
    ana = m.analyze(text)
    adj = ''
    for word in ana:
        if 'analysis' in word:
            gr = word['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            lex = word['text']
            if adj == '':
                if pos == 'A':
                    adj = lex
            else:
                if pos == 'S':
                    print(adj, '', lex)
                    adj = ''
                elif pos!= 'A':
                    adj = ''
                else:
                    adj = lex

def ne_verb(text):
    ana = m.analyze(text)
    ne = ''
    for word in ana:
        if 'analysis' in word:
            gr = word['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            lex = word['text']
            if ne == '':
                if lex == 'не':
                    ne = lex
            else:
                if pos == 'V':
                    print(ne, '', lex)
                    ne = ''
                elif lex!= 'не':
                    ne = ''
                else:
                    ne = lex

def noun_adj(text):
    ana = m.analyze(text)
    noun = ''
    for word in ana:
        if 'analysis' in word:
            gr = word['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            lex = word['text']
            if noun == '':
                if pos == 'S':
                    noun = lex
            else:
                if pos == 'A':
                    print(noun, '', lex)
                    ne = ''
                elif lex!= 'S':
                    noun = ''
                else:
                    noun = lex

In [114]:
accuracy_check_rus(mystem(text))

Accuracy: 0.8529


In [115]:
accuracy_check_rus(nat(text))

Accuracy: 0.8137


In [116]:
accuracy_check_rus(pymorph(text))

Accuracy: 0.7941


In [117]:
print(pymorph(text))

['NOUN', 'VERB', 'ADJ', 'ADJ', 'ADJ', 'VERB', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'PREP', 'NOUN', 'ADJ', 'VERB', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'PREP', 'NOUN', 'VERB', 'NOUN', 'NOUN', 'VERB', 'ADV', 'NOUN', 'NOUN', 'PART', 'VERB', 'PREP', 'NOUN', 'PART', 'NOUN', 'VERB', 'NOUN', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'VERB', 'ADV', 'NOUN', 'NOUN', 'ADJ', 'VERB', 'ADV', 'ADV', 'VERB', 'NOUN', 'VERB', 'CONJ', 'VERB', 'ADJ', 'NOUN', 'ADJ', 'ADJ', 'ADV', 'VERB', 'PRO', 'CONJ', 'ADV', 'ADV', 'ADV', 'ADJ', 'PREP', 'None', 'NOUN', 'NOUN', 'CONJ', 'PRO', 'PART', 'VERB', 'CONJ', 'ADV', 'VERB', 'CONJ', 'ADV', 'VERB', 'NOUN', 'CONJ', 'PREP', 'NUMR', 'NOUN', 'VERB', 'VERB']


In [118]:
print(mystem(text))

['NOUN', 'VERB', 'ADJ', 'ADJ', 'ADV', 'VERB', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'PREP', 'NOUN', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'VERB', 'NOUN', 'PREP', 'NOUN', 'ADJ', 'NOUN', 'PREP', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'PREP', 'NOUN', 'VERB', 'NOUN', 'NOUN', 'VERB', 'NOUN', 'NOUN', 'NOUN', 'PART', 'VERB', 'PREP', 'NOUN', 'PART', 'NOUN', 'VERB', 'NOUN', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'NOUN', 'ADJ', 'VERB', 'ADV', 'ADV', 'VERB', 'NOUN', 'VERB', 'CONJ', 'VERB', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'ADV', 'VERB', 'PRO', 'VERB', 'ADV', 'ADV', 'ADV', 'ADJ', 'PREP', 'None', 'NOUN', 'NOUN', 'CONJ', 'PRO', 'PART', 'VERB', 'CONJ', 'ADV', 'VERB', 'CONJ', 'ADV', 'VERB', 'NOUN', 'CONJ', 'PREP', 'NUMR', 'NOUN', 'VERB', 'VERB']


In [119]:
print(nat(text))

['NOUN', 'VERB', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'NOUN', 'PREP', 'NOUN', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'VERB', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'X', 'ADJ', 'NOUN', 'PREP', 'NOUN', 'VERB', 'NOUN', 'NOUN', 'VERB', 'NOUN', 'NOUN', 'NOUN', 'PART', 'VERB', 'PREP', 'NOUN', 'ADV', 'NOUN', 'VERB', 'NOUN', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'NOUN', 'ADJ', 'VERB', 'ADV', 'ADV', 'VERB', 'NOUN', 'VERB', 'CONJ', 'VERB', 'PRO', 'NOUN', 'ADJ', 'NOUN', 'ADJ', 'ADJ', 'PRON', 'VERB', 'ADV', 'ADV', 'ADV', 'ADJ', 'PREP', 'NOUN', 'ADJ', 'NOUN', 'CONJ', 'PRON', 'PART', 'VERB', 'CONJ', 'ADV', 'VERB', 'CONJ', 'ADV', 'VERB', 'NOUN', 'CONJ', 'PREP', 'NUM', 'NOUN', 'VERB', 'VERB']


In [120]:
print(spacy_tag(text_eng))

['NOUN', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'VERB', 'PREP', 'DET', 'NOUN', 'VERB', 'ADJ', 'PREP', 'DET', 'ADJ', 'NOUN', 'PRON', 'VERB', 'VERB', 'ADJ', 'NOUN', 'CONJ', 'ADJ', 'NOUN', 'DET', 'NOUN', 'VERB', 'VERB', 'ADV', 'PREP', 'ADJ', 'NOUN', 'CONJ', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'PREP', 'DET', 'NOUN', 'PRON', 'VERB', 'NOUN', 'PREP', 'DET', 'NOUN', 'PRON', 'VERB', 'DET', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'PREP', 'DET', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'ADV', 'ADJ', 'NOUN', 'VERB', 'PREP', 'DET', 'NOUN', 'VERB', 'DET', 'NOUN', 'PRON', 'VERB', 'VERB', 'PREP', 'DET', 'NOUN', 'PRON', 'VERB', 'DET', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'PRON', 'VERB', 'VERB', 'NOUN', 'VERB', 'ADJ', 'PREP', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'DET', 'NOUN', 'PREP', 'DET', 'NOUN', 'PREP', 'DET', 'NOUN']


In [121]:
print(flair_tag(text_eng))

['NOUN', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'VERB', 'PREP', 'DET', 'NOUN', 'VERB', 'ADJ', 'PREP', 'DET', 'ADJ', 'NOUN', 'PRON', 'VERB', 'VERB', 'ADJ', 'NOUN', 'CONJ', 'ADJ', 'NOUN', 'DET', 'NOUN', 'VERB', 'VERB', 'ADV', 'PREP', 'ADJ', 'NOUN', 'CONJ', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'PREP', 'DET', 'NOUN', 'PRON', 'VERB', 'NOUN', 'PREP', 'DET', 'NOUN', 'PRON', 'VERB', 'DET', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'PREP', 'DET', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'ADV', 'ADJ', 'NOUN', 'VERB', 'PREP', 'DET', 'NOUN', 'VERB', 'PRON', 'NOUN', 'PRON', 'VERB', 'VERB', 'PREP', 'DET', 'NOUN', 'PRON', 'VERB', 'PRON', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'PRON', 'VERB', 'VERB', 'NOUN', 'VERB', 'ADJ', 'PREP', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'PRON', 'NOUN', 'PREP', 'DET', 'NOUN', 'PREP', 'DET', 'NOUN']


In [122]:
print(nltk_tags(text_eng))

['NOUN', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'VERB', 'PREP', 'DET', 'NOUN', 'VERB', 'ADJ', 'PREP', 'DET', 'ADJ', 'NOUN', 'PRO', 'VERB', 'VERB', 'NOUN', 'NOUN', 'CONJ', 'ADJ', 'NOUN', 'DET', 'NOUN', 'VERB', 'VERB', 'ADV', 'PREP', 'ADJ', 'NOUN', 'CONJ', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'PREP', 'DET', 'NOUN', 'PRO', 'VERB', 'NOUN', 'PREP', 'DET', 'NOUN', 'PRO', 'VERB', 'DET', 'NOUN', 'NOUN', 'VERB', 'VERB', 'PREP', 'DET', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'ADV', 'ADJ', 'NOUN', 'VERB', 'PREP', 'DET', 'NOUN', 'VERB', 'PRO', 'NOUN', 'PRO', 'VERB', 'VERB', 'PREP', 'DET', 'NOUN', 'PRO', 'VERB', 'PRO', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'PRO', 'VERB', 'VERB', 'NOUN', 'VERB', 'ADJ', 'PREP', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'PRO', 'NOUN', 'PREP', 'DET', 'NOUN', 'PREP', 'DET', 'NOUN']


In [123]:
accuracy_check_eng(spacy_tag(text_eng))

Accuracy: 0.8725


In [124]:
accuracy_check_eng(flair_tag(text_eng))

Accuracy: 0.8725


In [125]:
accuracy_check_eng(nltk_tags(text_eng))

Accuracy: 0.9412


Чтобы улучшить работы программы из предыдущей дз, предлагаю выделить следующие синтаксические группы: сущ + прил и прил + сущ, т.к достаточно часто в зависимости от рецензии сторонам фильма даётся характеристика: плохая режиссура, саундтрек великолепен и т.д. Для только отрицательных рецензий можно выделить не + глаг., так как при описании впечатлений автор может перечислять то, что он ожидал от фильма, но ему не удалось реализовать: не впечатлил, не заинтересовал, не понравился и т.д.

In [126]:
review = 'Ремейк «Мулан» от Дисней — явление само по себе интересное. Прошлые реинкарнации \
анимационного наследия студии — вроде фавровских «Короля льва» с  \
«Книгой джунглей» и «Аладдина» Гая Ричи — продвигали в первую очередь как старые \
истории на новый лад, как способ ещё раз, но в другой форме, пережить «тот самый» \
детский экспириенс. С «Мулан» всё несколько сложнее: видите ли, оригинальный мультфильм, \
рассказывающий о народной китайской героине, понравился примерно всем, кроме самих китайцев. \
Потому что и дракон Мушу — персонаж оскорбительный (негоже величественным драконам быть смехотворными), \
и сама Мулан (которая вообще-то Хуа Мулань) вдруг стала индивидуалисткой-феминисткой, а не \
преданной защитницей Китайской Империи и китайской национальной идеи. В 1998-м недовольство \
одной отдельной закрытой страны, допустим, легко можно было пережить. В 2020-м продюсерам \
Дисней легче вынести отсутствие домашнего проката, чем невыпуск кино в Китае. \
Поэтому и основной опорой в маркетинге «Мулан» была не эксплуатация ностальгии, а, наоборот, \
всяческие увещевания, что экранизация совершенно не будет походить на мультфильм. Что она будет \
более приземлённой, историчной и суровой — якобы поэтому и Мушу, персонажу явно фэнтезийному, места \
в ней не нашлось. Реальность оказалась несколько более прозаичной: ни о какой приземлённости и \
историчности в новой «Мулан», как и в оригинальном мультфильме, речи не идёт — не стала свежая \
экранизация и ближе к изначальной легенде о Хуа Мулань. Магии здесь стало даже больше: есть и \
ведьмы, и фениксы, да и силы главной героини объясняются не упорством и трудом, а предрасположенностью \
к этакой Силе из «Звёздных войн» (точнее, взаимосвязь тут обратная, но неважно). \
Если вы ищете приземлённую историю героини — так её уже сняли сами китайцы в 2009, \
Хуа Мулан называется. А это просто ещё одна сказка, только со слегка сбившейся в \
сторону коммунизма оптикой.'
                
    
       

In [127]:
adj_noun(review)

Прошлые  реинкарнации
анимационного  наследия
старые  истории
новый  лад
детский  экспириенс
оригинальный  мультфильм
китайской  героине
величественным  драконам
преданной  защитницей
Китайской  Империи
национальной  идеи
домашнего  проката
основной  опорой
всяческие  увещевания
фэнтезийному  места
новой  Мулан
оригинальном  мультфильме
свежая  экранизация
изначальной  легенде
главной  героини
Звёздных  войн
приземлённую  историю


In [128]:
noun_adj(review)

реинкарнации  анимационного
персонаж  оскорбительный
защитницей  Китайской
отсутствие  домашнего
силы  главной


In [129]:
ne_verb(review)

не  будет
не  нашлось
не  идёт
не  стала
